In [ ]:
# эти файлы лежат внутри purchases.zip и zakupki.zip
# внутри purchaseNotice.7z и purchaseNotice_daily.7z лежат куча xml файлов,
# относящихся к СПБ, у всех файлов одна и та же структура
!gdown 1LU-XcPo1FGK33Le3Xka_N3EoSprPouT5
!gdown 1HGTDFau5O0yUFsfEvWLA2C1NVTf5ATdl
!7z x purchaseNotice.7z
!7z x purchaseNotice_daily.7z

first_folder = '/content/purchaseNotice'
second_folder = '/content/purchaseNotice_daily'

Downloading...
From: https://drive.google.com/uc?id=1LU-XcPo1FGK33Le3Xka_N3EoSprPouT5
To: /content/purchaseNotice.7z
100% 726M/726M [00:07<00:00, 101MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HGTDFau5O0yUFsfEvWLA2C1NVTf5ATdl
To: /content/purchaseNotice_daily.7z
100% 1.05G/1.05G [00:09<00:00, 112MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 725671033 bytes (693 MiB)

Extracting archive: purchaseNotice.7z
--
Path = purchaseNotice.7z
Type = 7z
Physical Size = 725671033
Headers Size = 19313
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 1        0% 8 - purchaseNotice/purchaseNotice_ . 000_20170131_235959_008.xml.zip                               

In [ ]:
!pip install lxml beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from zipfile import ZipFile, BadZipFile

from bs4 import BeautifulSoup

from collections import defaultdict
from tqdm import tqdm
import pandas as pd

In [ ]:
# слева указаны, какие поля будут в датафрейме, справа поля в xml файле, если справа список,
# то это указывает на путь, то есть в xml ищется вершина из начала списка, внутри вершина на
# втором месте и т.д.
key_xml = {
    'guid': 'ns2:guid',
    'registrationNumber': 'ns2:registrationNumber',
    'name': 'ns2:name',
    'termination': 'ns2:termination',
    'urlEIS': 'ns2:urlEIS',
    'createDateTime': 'ns2:createDateTime',
    'publicationDateTime': 'ns2:publicationDateTime',
    'applSubmisionPlace': 'ns2:applSubmisionPlace',
    'submissionCloseDateTime': 'ns2:submissionCloseDateTime',
    'customer_fullName': ['ns2:customer', 'mainInfo', 'fullName'],
    'customer_shortName': ['ns2:customer', 'mainInfo', 'shortName'],
    'customer_inn': ['ns2:customer', 'mainInfo', 'inn'],
    'placer_fullName': ['ns2:placer', 'mainInfo', 'fullName'],
    'placer_shortName': ['ns2:placer', 'mainInfo', 'shortName'],
    'placer_inn': ['ns2:placer', 'mainInfo', 'inn'],
    'initialSum': 'initialSum',
    'currency_code': ['currency', 'code'],
    'subject': 'subject',
}

In [ ]:
# внутри 7z архива находятся ещё zip архивы, их тоже надо извлечь
out_folder = '1_out'
os.makedirs(out_folder, exist_ok=True)

for filename in tqdm(os.listdir(first_folder)):
    try:
        with ZipFile(f"{first_folder}/{filename}", 'r') as zObject:
            zObject.extractall(out_folder)
    except BadZipFile as e:
        pass

100%|██████████| 1743/1743 [01:14<00:00, 23.26it/s]


In [ ]:
# излвечение данных из xml файлов с помощью beatifulsoup, какие данные извлекаются
# указано в одной из ячеек выше

zakupki_data = defaultdict(list)

for fn in tqdm(os.listdir(out_folder)):
    filepath = f"{out_folder}/{fn}"

    with open(filepath, 'r') as f:
        data = f.read()

    if len(data) == 0:
        continue

    soup = BeautifulSoup(data, 'xml')

    for item in soup.find_all('ns2:item'):
        for k, path in key_xml.items():
            try:
                if isinstance(path, str):
                    zakupki_data[k].append(item.find(path).text)
                else:
                    item_ = item
                    for p in path:
                        item_ = item_.find(p)
                    zakupki_data[k].append(item_.text)
            except AttributeError:
                zakupki_data[k].append(None)

zakupki_df = pd.DataFrame(zakupki_data)

100%|██████████| 11285/11285 [03:51<00:00, 48.82it/s]


In [ ]:
# тоже самое, но для файла purchaseNotice_daily
out2_folder = '2_out'
os.makedirs(out2_folder, exist_ok=True)

for filename in tqdm(os.listdir(second_folder)):
    try:
        with ZipFile(f"{second_folder}/{filename}", 'r') as zObject:
            zObject.extractall(out2_folder)
    except BadZipFile as e:
        pass

100%|██████████| 4880/4880 [02:05<00:00, 38.76it/s]


In [ ]:
contract_data = defaultdict(list)

for fn in tqdm(os.listdir(out2_folder)):
    filepath = f"{out2_folder}/{fn}"
    try:
        with open(filepath, 'r') as f:
            data = f.read()
    except UnicodeDecodeError:
        continue

    if len(data) == 0:
        continue

    soup = BeautifulSoup(data, 'xml')

    for item in soup.find_all('ns2:item'):
        for k, path in key_xml.items():
            try:
                if isinstance(path, str):
                    contract_data[k].append(item.find(path).text)
                else:
                    item_ = item
                    for p in path:
                        item_ = item_.find(p)
                    contract_data[k].append(item_.text)
            except AttributeError:
                contract_data[k].append(None)

zakupki_df = pd.concat([zakupki_df, pd.DataFrame(contract_data)]).drop_duplicates('registrationNumber')

100%|██████████| 195958/195958 [1:21:35<00:00, 40.03it/s]


In [ ]:
len(zakupki_df)

165186

In [ ]:
zakupki_df.to_csv('/content/drive/MyDrive/вкр/purchases.csv', sep='|', index=None)